In [1]:
import pytorch_lightning as ptl
import polars as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model.peephole_lstm import PeepholeLSTM
from model.data_loader import load_data

In [3]:
data: pl.DataFrame = load_data("/home/pupperemeritus/isro_project/data/October2023.parquet")

In [5]:
data.head(20)

GPS_WN,GPS_TOW,SVID,Value of the RxState field of the ReceiverStatus SBF block,Azimuth,Elevation,Average Sig1 C/N0 over the last minute (dB-Hz),S4,Correction to total S4 on Sig1 (thermal noise component only) (dimensionless),"Phi01 on Sig1, 1-second phase sigma (radians)","Phi03 on Sig1, 3-second phase sigma (radians)","Phi10 on Sig1, 10-second phase sigma (radians)","Phi30 on Sig1, 30-second phase sigma (radians)","Phi60 on Sig1, 60-second phase sigma (radians)","AvgCCD on Sig1, average of code/carrier divergence (meters)","SigmaCCD on Sig1, standard deviation of code/carrier divergence (meters)",TEC at TOW - 45 seconds (TECU),dTEC from TOW - 60s to TOW - 45s (TECU),TEC at TOW - 30 seconds (TECU),dTEC from TOW - 45s to TOW - 30s (TECU),TEC at TOW - 15 seconds (TECU),dTEC from TOW - 30s to TOW - 15s (TECU),TEC at TOW (TECU),dTEC from TOW - 15s to TOW (TECU),Sig1 lock time (seconds),sbf2ismr version number,Lock time on the second frequency used for the TEC computation (seconds),Averaged C/N0 of second frequency used for the TEC computation (dB-Hz),SI Index on Sig1: (10*log10(Pmax)-10*log10(Pmin))/(10*log10(Pmax)+10*log10(Pmin)) (dimensionless),"SI Index on Sig1, numerator only: 10*log10(Pmax)-10*log10(Pmin) (dB)","p on Sig1, spectral slope of detrended phase in the 0.1 to 25Hz range (dimensionless)",Average Sig2 C/N0 over the last minute (dB-Hz),Total S4 on Sig2 (dimensionless),Correction to total S4 on Sig2 (thermal noise component only) (dimensionless),"Phi01 on Sig2, 1-second phase sigma (radians)","Phi03 on Sig2, 3-second phase sigma (radians)","Phi10 on Sig2, 10-second phase sigma (radians)","Phi30 on Sig2, 30-second phase sigma (radians)","Phi60 on Sig2, 60-second phase sigma (radians)","AvgCCD on Sig2, average of code/carrier divergence (meters)","SigmaCCD on Sig2, standard deviation of code/carrier divergence (meters)",Sig2 lock time (seconds),SI Index on Sig2 (dimensionless),"SI Index on Sig2, numerator only (dB)","p on Sig2, phase spectral slope in the 0.1 to 25Hz range (dimensionless)",Average Sig3 C/N0 over the last minute (dB-Hz),Total S4 on Sig3 (dimensionless),Correction to total S4 on Sig3 (thermal noise component only) (dimensionless),"Phi01 on Sig3, 1-second phase sigma (radians)","Phi03 on Sig3, 3-second phase sigma (radians)","Phi10 on Sig3, 10-second phase sigma (radians)","Phi30 on Sig3, 30-second phase sigma (radians)","Phi60 on Sig3, 60-second phase sigma (radians)","AvgCCD on Sig3, average of code/carrier divergence (meters)","SigmaCCD on Sig3, standard deviation of code/carrier divergence (meters)",Sig3 lock time (seconds),SI Index on Sig3 (dimensionless),"SI Index on Sig3, numerator only (dB)","p on Sig3, phase spectral slope in the 0.1 to 25Hz range (dimensionless)","T on Sig1, phase power spectral density at 1 Hz (rad^2/Hz)","T on Sig2, phase power spectral density at 1 Hz (rad^2/Hz)","T on Sig3, phase power spectral density at 1 Hz (rad^2/Hz)",Latitude,Longitude,IST_Time
i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[μs]
2282,60,5,630,17,67,49.4,0.027,0.034,0.017,0.031,0.04,0.04,0.041,-14.596,0.004,12.61,0.01,12.572,0.007,12.591,0.008,12.572,0.016,13911,781,7834,57.4,0.007,0.782,1.8,48.7,0.035,0.037,0.017,0.03,0.039,0.039,0.04,-25.402,0.007,7834,0.01,0.93,1.66,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.000027,0.00003,null,18.777651,78.758218,2023-10-01 05:31:00
2282,60,11,630,47,20,36.2,0.124,0.156,0.039,0.041,0.043,0.043,0.043,-30.136,0.018,19.332,0.089,19.37,0.118,19.77,0.109,19.989,0.078,22104,781,22100,22.1,0.039,3.334,1.31,39.5,0.109,0.106,0.036,0.042,0.054,0.056,0.058,-48.513,0.05,256,0.038,2.975,1.51,42.2,0.079,0.077,0.027,0.037,0.056,0.06,0.069,-53.368,0.066,22143,0.026,2.3,1.45,0.000137,0.000148,0.000081,22.710547,84.609833,2023-10-01 05:31:00
228

In [6]:
data.describe()

statistic,GPS_WN,GPS_TOW,SVID,Value of the RxState field of the ReceiverStatus SBF block,Azimuth,Elevation,Average Sig1 C/N0 over the last minute (dB-Hz),S4,Correction to total S4 on Sig1 (thermal noise component only) (dimensionless),"Phi01 on Sig1, 1-second phase sigma (radians)","Phi03 on Sig1, 3-second phase sigma (radians)","Phi10 on Sig1, 10-second phase sigma (radians)","Phi30 on Sig1, 30-second phase sigma (radians)","Phi60 on Sig1, 60-second phase sigma (radians)","AvgCCD on Sig1, average of code/carrier divergence (meters)","SigmaCCD on Sig1, standard deviation of code/carrier divergence (meters)",TEC at TOW - 45 seconds (TECU),dTEC from TOW - 60s to TOW - 45s (TECU),TEC at TOW - 30 seconds (TECU),dTEC from TOW - 45s to TOW - 30s (TECU),TEC at TOW - 15 seconds (TECU),dTEC from TOW - 30s to TOW - 15s (TECU),TEC at TOW (TECU),dTEC from TOW - 15s to TOW (TECU),Sig1 lock time (seconds),sbf2ismr version number,Lock time on the second frequency used for the TEC computation (seconds),Averaged C/N0 of second frequency used for the TEC computation (dB-Hz),SI Index on Sig1: (10*log10(Pmax)-10*log10(Pmin))/(10*log10(Pmax)+10*log10(Pmin)) (dimensionless),"SI Index on Sig1, numerator only: 10*log10(Pmax)-10*log10(Pmin) (dB)","p on Sig1, spectral slope of detrended phase in the 0.1 to 25Hz range (dimensionless)",Average Sig2 C/N0 over the last minute (dB-Hz),Total S4 on Sig2 (dimensionless),Correction to total S4 on Sig2 (thermal noise component only) (dimensionless),"Phi01 on Sig2, 1-second phase sigma (radians)","Phi03 on Sig2, 3-second phase sigma (radians)","Phi10 on Sig2, 10-second phase sigma (radians)","Phi30 on Sig2, 30-second phase sigma (radians)","Phi60 on Sig2, 60-second phase sigma (radians)","AvgCCD on Sig2, average of code/carrier divergence (meters)","SigmaCCD on Sig2, standard deviation of code/carrier divergence (meters)",Sig2 lock time (seconds),SI Index on Sig2 (dimensionless),"SI Index on Sig2, numerator only (dB)","p on Sig2, phase spectral slope in the 0.1 to 25Hz range (dimensionless)",Average Sig3 C/N0 over the last minute (dB-Hz),Total S4 on Sig3 (dimensionless),Correction to total S4 on Sig3 (thermal noise component only) (dimensionless),"Phi01 on Sig3, 1-second phase sigma (radians)","Phi03 on Sig3, 3-second phase sigma (radians)","Phi10 on Sig3, 10-second phase sigma (radians)","Phi30 on Sig3, 30-second phase sigma (radians)","Phi60 on Sig3, 60-second phase sigma (radians)","AvgCCD on Sig3, average of code/carrier divergence (meters)","SigmaCCD on Sig3, standard deviation of code/carrier divergence (meters)",Sig3 lock time (seconds),SI Index on Sig3 (dimensionless),"SI Index on Sig3, numerator only (dB)","p on Sig3, phase spectral slope in the 0.1 to 25Hz range (dimensionless)","T on Sig1, phase power spectral density at 1 Hz (rad^2/Hz)","T on Sig2, phase power spectral density at 1 Hz (rad^2/Hz)","T on Sig3, phase power spectral density at 1 Hz (rad^2/Hz)",Latitude,Longitude,IST_Time
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""",2.861526e6,2.861526e6,2.861526e6,2.861526e6,2.861526e6,2.861526e6,2.861526e6,2.861526e6,2.861526e6,2.861193e6,2.86084e6,2.859392e6,2.85588e6,2.851929e6,2.861526e6,2.861526e6,2.04812e6,2.04812e6,2.047947e6,2.03574e6,2.047866e6,2.035561e6,2.064552e6,2.047866e6,2.856186e6,2.861526e6,2.100334e6,2.141623e6,2.861525e6,2.861525e6,2.848677e6,2.05852e6,2.057873e6,2.057873e6,2.057133e6,2.056179e6,2.05108e6,2.03582e6,2.019739e6,2.058519e6,2.058519e6,2.018553e6,2.058772e6,2.058772e6,1.99261e6,1.854944e6,1.854909e6,1.854909e6,1.854717e6,1.854553e6,1.854001e6,1.852861e6,1.851745e6,1.854943e6,1.854943e6,1.854101e6,1.854951e6,1.854951e6,1.850672e6,2.848677e6,1.99261e6,1.850672e6,2.861526e6,2.861526e6,"""2861526"""
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
class PeepholeLSTMRegressionModule(ptl.LightningModule):
    def __init__(
        self, input_size, hidden_size, num_layers, output_size, learning_rate=0.001
    ):
        super().__init__()
        self.save_hyperparameters()

        self.lstm = PeepholeLSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_size)
        lstm_out, _ = self.lstm(x)
        # We'll use the output of the last time step
        out = self.fc(lstm_out[:, -1, :])
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log("test_loss", loss)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

In [ ]:
class PLSTM(ptl.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers):
        self.lstm = PeepholeLSTM()